In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("traffic-density-w6zd9").project("traffic-density-izacs-uhqed")
version = project.version(3)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-Density-3 in yolov8:: 100%|██████████| 33418/33418 [00:04<00:00, 7079.26it/s]


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 34.7 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv, C2f

# Revised TGATLayer with Static Image Support
class TGATLayer(nn.Module):
    def __init__(self, channels, num_heads=4, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(channels, num_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(channels, channels * 4), 
            nn.GELU(),
            nn.Linear(channels * 4, channels)
        )
        self.norm1 = nn.LayerNorm(channels)
        self.norm2 = nn.LayerNorm(channels)
        
    def forward(self, x):
        B, C, H, W = x.shape
        x_flat = x.view(B, C, -1).permute(0, 2, 1)  # [B, N, C]
        
        # Self-attention
        attn_out, _ = self.attention(x_flat, x_flat, x_flat)
        x_flat = self.norm1(x_flat + attn_out)
        
        # FFN
        ffn_out = self.ffn(x_flat)
        x_flat = self.norm2(x_flat + ffn_out)
        
        return x_flat.permute(0, 2, 1).view(B, C, H, W)

# Revised TGATNeck with Channel Management
class TGATNeck(nn.Module):
    def __init__(self, backbone_channels=[256, 512, 1024], num_heads=4):
        super().__init__()
        
        # Top-down path
        self.lateral_conv5 = Conv(backbone_channels[2], backbone_channels[1], 1)
        self.lateral_conv4 = Conv(backbone_channels[1], backbone_channels[0], 1)
        
        # TGAT blocks
        self.tgat5 = TGATLayer(backbone_channels[1], num_heads)
        self.tgat4 = TGATLayer(backbone_channels[0], num_heads)
        self.tgat3 = TGATLayer(backbone_channels[0], num_heads)
        
        # Bottom-up path
        self.up_conv3 = Conv(backbone_channels[0], backbone_channels[1], 3)
        self.up_conv4 = Conv(backbone_channels[1], backbone_channels[2], 3)

    def forward(self, features):
        c3, c4, c5 = features
        
        # Top-down
        p5 = self.lateral_conv5(c5)
        p4 = self.lateral_conv4(c4 + F.interpolate(p5, scale_factor=2))
        p3 = self.tgat3(c3 + F.interpolate(p4, scale_factor=2))
        
        # Bottom-up
        n3 = p3
        n4 = self.tgat4(p4 + self.up_conv3(F.max_pool2d(n3, 2)))
        n5 = self.tgat5(p5 + self.up_conv4(F.max_pool2d(n4, 2)))
        
        return [n3, n4, n5]

# Robust YOLOv8 Integration
class TGATYOLO(YOLO):
    def __init__(self, cfg='yolov8s.yaml', num_heads=4):
        super().__init__(cfg)
        self.replace_neck(num_heads)
        
    def replace_neck(self, num_heads):
        # Get actual backbone channels from C2f modules
        backbone_channels = [
            self.model.model[i].cv2.conv.out_channels  # Correct channel access
            for i in [4, 6, 8]  # These indices work for YOLOv8s v8.0.xx
        ]
        
        # Replace neck module (index -2 for most YOLOv8 versions)
        self.model.model[-2] = TGATNeck(backbone_channels, num_heads)

# Training Configuration
if __name__ == '__main__':
    model = TGATYOLO('yolov8s.yaml')
    model.train(
        data='/kaggle/working/Traffic-Density-3/data.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        lr0=0.01,
        lrf=0.1,
        optimizer="SGD",
        augment=True,
        mosaic=True,
        weight_decay=0.0005,
        dropout=0.1  # Match dropout from TGATLayer
    )

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/kaggle/working/Traffic-Density-3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False

100%|██████████| 755k/755k [00:00<00:00, 47.4MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 193MB/s]


AMP: checks passed ✅


...

                   all       1811       9377      0.703      0.747      0.771      0.497

50 epochs completed in 3.918 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8s summary (fused): 72 layers, 11,128,680 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:28<00:00,  2.03it/s]


                   all       1811       9377      0.696      0.744      0.771      0.494
               Bicycle        398        511      0.776      0.957      0.954      0.546
                E-bike        504        516      0.916      0.978      0.989      0.691
               Jeepney        837       1592      0.707      0.596       0.72      0.448
            Motorcycle        694       1556      0.526      0.506      0.535      0.269
            Pedestrian        568       1449      0.527      0.428      0.498      0.246
              Tricycle        684        882      0.882      0.853       0.92      0.628
                 Truck        728        882      0.768      0.868      0.895      0.653
               Vehicle        494       1989      0.465      0.763      0.653      0.471


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 11.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train
